内容来自 [七年实现财富自由|雪球](https://xueqiu.com/u/1164950442)

In [6]:
import pandas as pd
import numpy as np

from empyrical import max_drawdown

产生如下报错：
```bash
ModuleNotFoundError: No module named 'empyrical'
```

> 需要安装 empyrical 包

```bash
! pip install empyrical-reloaded --upgrade --user -i https://pypi.tuna.tsinghua.edu.cn/simple
```

In [2]:
symbols = ['513100','513500']#,'510500','159915','510880'
symbols = {'513100':'纳指100',
           '513500':'标普500',
           '510300':'沪深300',
           '159915':'创业板',
           '518880':'黄金',
           #'512890':'红利低波',
           #'159985':'豆粕',
           '511880':'银华日利-货币ETF'
          }

dfs = []
for s in symbols.keys():
    df = pd.read_csv(f'data/{s}.csv')
    df['symbol'] = s
    dfs.append(df)
    df_all = pd.concat(dfs,axis=0)
    
df_all

,high,low,close,open,volume,date,symbol
0,0.999,0.989,0.997,0.990,877116,2013-05-15,513100
1,0.999,0.994,0.999,0.997,265570,2013-05-16,513100
2,1.000,0.996,0.997,0.999,36379,2013-05-17,513100
3,1.004,0.996,0.997,1.002,80490,2013-05-20,513100
4,0.998,0.994,0.997,0.997,161124,2013-05-21,513100
...,...,...,...,...,...,...,...
2974,132.821,132.812,132.821,132.813,1071268,2025-07-15,511880
2975,132.827,132.821,132.827,132.823,1234589,2025-07-16,511880
2976,132.832,132.827,132.830,132.829,1004811,2025-07-17,511880
2977,132.838,132.830,132.833,132.838,1605493,2025-07-18,511880


# 时间序列分析
## 大类资产ETF数据

In [3]:
df_close = pd.pivot(df_all,columns='symbol',index='date',values='close')
new_cols = {}
for c in df_close.columns:
    new_cols[c] = symbols[c]
print(new_cols)
df_close.rename(columns = new_cols,inplace=True)
df_close.index  = pd.to_datetime(df_close.index)
df_close = df_close.dropna() #对齐数据
df_close = df_close.ffill()  # 前向填充
df_close

# 注释掉我自己写的代码
# # 先将date列转换为datetime格式
# df_all['date'] = pd.to_datetime(df_all['date'])
# df_close = df_all.pivot_table(values='close', index=df_all.date, columns='symbol')
# df_close.dropna(inplace=True)
# df_close

{'159915': '创业板', '510300': '沪深300', '511880': '银华日利-货币ETF', '513100': '纳指100', '513500': '标普500', '518880': '黄金'}


symbol,创业板,沪深300,银华日利-货币ETF,纳指100,标普500,黄金
date,,,,,,
2014-01-15,1.390,2.287,103.453,1.168,0.986,2.434
2014-01-16,1.371,2.288,103.495,1.181,0.991,2.437
2014-01-17,1.367,2.256,103.491,1.183,0.987,2.435
2014-01-20,1.342,2.245,103.459,1.180,0.982,2.459
2014-01-21,1.381,2.263,103.481,1.182,0.984,2.451
...,...,...,...,...,...,...
2025-07-15,2.212,4.835,132.821,8.240,4.262,7.436
2025-07-16,2.206,4.826,132.827,8.190,4.230,7.402
2025-07-17,2.247,4.859,132.830,8.250,4.262,7.397


### 计算年化收益

In [7]:
def calculate_cagr(price_series):
    """
    计算复合年增长率 (CAGR)

    参数:
    price_series: 价格序列 (pd.Series)，索引为日期

    返回:
    cagr: 复合年增长率 (小数形式)
    """

    # 获取第一个和最后一个价格
    start_price = price_series.iloc[0]
    end_price = price_series.iloc[-1]

    # 计算总天数
    days = (price_series.index[-1] - price_series.index[0]).days

    # 计算年数
    years = days / 365.0

    # 计算CAGR
    cagr = (end_price / start_price) ** (1 / years) - 1

    return cagr

cagr = calculate_cagr(df_close)
cagr

symbol
创业板           0.043693
沪深300         0.068643
银华日利-货币ETF    0.021935
纳指100         0.185683
标普500         0.135904
黄金            0.101922
dtype: float64

### 计算夏普比

In [8]:
def calculate_sharpe_ratio(return_series, risk_free_rate=0.0, periods=252):
    """
    计算年化夏普比率

    参数:
    return_series: 日收益率序列 (pd.Series)
    risk_free_rate: 年化无风险利率 (默认0)
    periods: 年化因子 (默认252个交易日)

    返回:
    sharpe_ratio: 年化夏普比率
    """
    # 计算平均日收益率
    mean_daily_return = return_series.mean()

    # 计算日收益率标准差
    std_daily_return = return_series.std()

    # 年化收益率
    annualized_return = (1 + mean_daily_return) ** periods - 1

    # 年化波动率
    annualized_volatility = std_daily_return * np.sqrt(periods)

    # 计算夏普比率
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

    return sharpe_ratio

# 计算夏普比率（假设无风险利率为3%）
sharpe = calculate_sharpe_ratio(df_close.pct_change(), risk_free_rate=0.0)
sharpe

symbol
创业板           0.315489
沪深300         0.450807
银华日利-货币ETF    6.629394
纳指100         1.024959
标普500         0.894289
黄金            0.881288
dtype: float64

### 计算最大回撤

In [9]:
mdd = max_drawdown(df_close.pct_change())
mdd.index = df_close.columns

mdd

symbol
创业板          -0.695778
沪深300        -0.443748
银华日利-货币ETF   -0.004077
纳指100        -0.285680
标普500        -0.296722
黄金           -0.204660
dtype: float64